In [4]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error as mae
from sklearn.metrics import mean_squared_error as mse
from sklearn.metrics import r2_score
from catboost import CatBoostRegressor

In [3]:
df=pd.read_excel('D:\Стас\Ученье - Свет\Оценка\Аспирантура\Видео по NN, Data mining, ABC и т.д\Распределение Гаусса, регрессия\Python\!Наработки\Квартиры Киров/Квартиры_Киров_выбока_2.xlsx', sheet_name = 0)
df.head()

,Комнатность,LN(Общая площадь),Общая площадь,Площадь кухни,Жилая площадь,Этаж 1,Этаж 2,Этажей в доме,Ремонт,Материал стен,...,последний,средний,деревянный,кирпичный,монолитный,панельный,дизайнерский,евро,косметический,требует ремонта
0,2к,3.799974,44.7,5.900000,29.458690,5,последний,5,евро,панельный,...,1,0,0,0,0,1,0,1,0,0
1,1к,3.465736,32.0,6.900000,17.655975,2,средний,5,косметический,кирпичный,...,0,1,0,1,0,0,0,0,1,0
2,студия,3.502550,33.2,6.633333,18.193506,3,последний,3,евро,кирпичный,...,1,0,0,1,0,0,0,1,0,0
3,2к,3.943522,51.6,8.800000,28.744021,6,средний,7,евро,кирпичный,...,0,1,0,1,0,0,0,1,0,0
4,3к,3.998201,54.5,18.000000,27.597229,18,средний,19,дизайнерский,панельный,...,0,1,0,0,0,1,1,0,0,0


In [5]:
df_v1 = df.copy()
len(df_v1)

2399

In [6]:
X = df_v1[['LN(Общая площадь)', 'Площадь кухни', 'Жилая площадь', 'Этажей в доме', 
           'Хронологический возраст', 'Долгота', 'Широта', 'Тип санузла 2', 
           '1к', '2к', '3к', '4к и более', 'студия', 'первый', 'последний', 'средний', 
           'деревянный', 'кирпичный', 'монолитный', 'панельный', 
           'дизайнерский', 'евро', 'косметический', 'требует ремонта']]

Y = df_v1['LN(Цена)']

# Делим имеющуюся выборку на обучающую и тестовую
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=0)

In [7]:
catb = CatBoostRegressor(iterations=2500, #максимальное количество построенных деревьев
                         random_seed=5, 
                         learning_rate=0.008, #скорость обучения, определяет насколько быстро или медленно модель будет учиться
                         max_depth=4, #глубина дерева
                         #l2_leaf_reg=0.01, #коэффициент при члене регуляризации L2 функции потерь
                         bagging_temperature=1,
                         random_strength=1,
                         #one_hot_max_size=8, #не используем one hot encoding, а включаем one_hot_max_size
                         loss_function='MAE',
                         od_type='Iter', #если модель переобучилась
                         od_wait=20) #делай остановку через 20 итераций
#если хотим хорошее качество, то уменьшаем learning_rate и увеличиваем iterations.
#если модель слишком рано переобучается, то l2_leaf_reg увеличиваем.

#cat_features = list(range(0, X.shape[1]))
#cat_features = list(df['transmission'])
#print(cat_features) 

catb.fit(X_train, Y_train, 
         #cat_features=cat_features, #массив с индексами категориальных фичей
         eval_set=(X_test, Y_test),
         verbose=False, 
         plot=True) #включение графика

Ypred = catb.predict(X_test)

pred_train = catb.predict(X_train)
mse_train = mse(Y_train, pred_train)

print(f'Train R2 {r2_score(Y_train, pred_train):.3f}')
#print(f'MSE на обучении {mse_train:.2f}')
print("MAE на обучении:", round(mae(Y_train, pred_train), 4))

pred_test = catb.predict(X_test)
mse_test = mse(Y_test, pred_test)

print(f'Test R2 {r2_score(Y_test, pred_test):.3f}')
#print(f'MSE на тесте {mse_test:.2f}')
print("MAE на тесте:", round(mae(Y_test, pred_test), 4))

MetricVisualizer(layout=Layout(align_self='stretch', height='500px'))

Train R2 0.932
MAE на обучении: 0.0944
Test R2 0.920
MAE на тесте: 0.1102


In [8]:
import pickle

In [9]:
pickle.dump(catb, open('cat_model.pkl', 'wb'))